In [1]:
from IPython.display import display

import numpy as np
import pandas as pd

from conf.settings import (
    get_settings, Settings,
    get_read_file_settings, ReadFileSettings,
)
from recsys_tasks.annotations.common import ProductInfoWithDetails
from recsys_tasks.processors.items_based import ItemBasedRecommendationsProcessor


In [2]:
# Получение конфигурации для расчетов.
settings: Settings = get_settings()
read_file_settings: ReadFileSettings = get_read_file_settings()

In [3]:
# Получение матричного представления исходных данных.
input_data_filename = '5_items_based.csv'
input_data_filepath = settings.INPUT_DATA_DIR / input_data_filename

source_data: np.ndarray = np.genfromtxt(
    input_data_filepath,
    dtype=float,
    **read_file_settings.model_dump(),
)
source_data: np.ndarray[tuple[int, int], int]
pd.DataFrame(source_data)

,0,1,2,3,4,5,6,7,8,9,10
0,4.0,3.0,2.0,3.0,4.0,5.0,3.0,2.0,4.0,4.0,NaN
1,3.0,4.0,3.0,5.0,3.0,4.0,3.0,3.0,NaN,2.0,NaN
2,5.0,5.0,3.0,4.0,4.0,3.0,3.0,4.0,NaN,4.0,NaN
3,4.0,4.0,3.0,3.0,1.0,4.0,1.0,3.0,3.0,2.0,NaN
4,4.0,5.0,3.0,2.0,2.0,2.0,2.0,3.0,4.0,3.0,NaN


In [4]:
def display_product_recommendations(products_list: list[ProductInfoWithDetails]):
    """Отображение детализации по продуктам рекомендаций."""
    for product in (products_list or []):
        print(
            f'Продукт №{product.product_id}: '
            f'рассчитанный рейтинг – {product.calculated_score}.'
        )
        
        if product.similar_products_similarity:
            print('Подобие используемых продуктов:')
            display(
                pd.DataFrame(
                    product.similar_products_similarity.matrix,
                    columns=product.similar_products_similarity.columns,
                    index=product.similar_products_similarity.index,
                )
            )
        if product.used_products_preferences:
            print('Оценки похожих продуктов от пользователя:')
            display(
                pd.DataFrame(
                    product.used_products_preferences.matrix,
                    columns=product.used_products_preferences.columns,
                    index=product.used_products_preferences.index,
                )
            )


def display_recommendation_results(rec_processor: ItemBasedRecommendationsProcessor):
    """Отображение результатов работы процессора."""
    results = rec_processor()
    
    print('Матрица подобия:')
    display(
        pd.DataFrame(
            rec_processor.similarity_matrix.matrix,
            columns=rec_processor.similarity_matrix.columns,
            index=rec_processor.similarity_matrix.index,
        )
    )
    
    print('Не оцененные пользователем продукты:', rec_processor.not_rated_products)
    
    if not rec_processor.is_new_user:
        print(
            'Минимальное пороговое значение для рекомендаций (средний рейтинг пользователя):',
            rec_processor.mean_user_score,
        )
    
    print()

    print('Рекомендованные продукты:')
    display_product_recommendations(results.recommended_products)

    print()
    
    print('Данные продукты не попадут в рекомендации:')
    display_product_recommendations(results.not_recommended_products)
    
    if rec_processor.is_new_user:
        print('\nДанный расчет был проведен для нового пользователя.')
        return


In [5]:
# Для существующего пользователя.
processor = ItemBasedRecommendationsProcessor(source_data, user_id=8, min_similarity_coefficient=0.85)
display_recommendation_results(processor)


Матрица подобия:


,0,1,2,3,4
0,NaN,0.872240,0.884841,0.908739,0.907007
1,0.872240,NaN,0.957025,0.890728,0.835306
2,0.884841,0.957025,NaN,0.887706,0.892681
3,0.908739,0.890728,0.887706,NaN,0.948683
4,0.907007,0.835306,0.892681,0.948683,NaN


Не оцененные пользователем продукты: [1, 2]
Минимальное пороговое значение для рекомендаций (средний рейтинг пользователя): 3.6666666666666665

Рекомендованные продукты:
Продукт №2: рассчитанный рейтинг – 3.6669306036281357.
Подобие используемых продуктов:


,2
0,0.884841
3,0.887706
4,0.892681


Оценки похожих продуктов от пользователя:


,8
0,4.0
3,3.0
4,4.0



Данные продукты не попадут в рекомендации:
Продукт №1: рассчитанный рейтинг – 3.4947567221340456.
Подобие используемых продуктов:


,1
0,0.872240
3,0.890728


Оценки похожих продуктов от пользователя:


,8
0,4.0
3,3.0


In [6]:
# Для нового пользователя.
processor = ItemBasedRecommendationsProcessor(source_data, user_id=10)
display_recommendation_results(processor)


Матрица подобия:


,0,1,2,3,4
0,NaN,0.872240,0.884841,0.908739,0.907007
1,0.872240,NaN,0.957025,0.890728,0.835306
2,0.884841,0.957025,NaN,0.887706,0.892681
3,0.908739,0.890728,0.887706,NaN,0.948683
4,0.907007,0.835306,0.892681,0.948683,NaN


Не оцененные пользователем продукты: [0, 1, 2, 3, 4]

Рекомендованные продукты:
Продукт №2: рассчитанный рейтинг – 3.888888888888889.

Данные продукты не попадут в рекомендации:

Данный расчет был проведен для нового пользователя.
